<a href="https://colab.research.google.com/gist/Mistobaan/f54c8b29985f4ec123c5ec4f9d03cbe6/eval-gsm8k-microsoft-phi-2-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overview

GSM-8k colab
Author [@fabmilo](https://twitter.com/fabmilo)

In [ ]:
!git clone https://github.com/EleutherAI/lm-evaluation-harness --depth 1
%cd lm-evaluation-harness
%pip install -e .

Cloning into 'lm-evaluation-harness'...
remote: Enumerating objects: 2631, done.
remote: Counting objects: 100% (2631/2631), done.
remote: Compressing objects: 100% (1770/1770), done.
remote: Total 2631 (delta 1035), reused 2058 (delta 843), pack-reused 0
Receiving objects: 100% (2631/2631), 1.04 MiB | 20.13 MiB/s, done.
Resolving deltas: 100% (1035/1035), done.
/content/lm-evaluation-harness
Obtaining file:///content/lm-evaluation-harness
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
%pip install transformers~=4.37.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 63.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

torch.set_default_device("cuda")

model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype="auto", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)

inputs = tokenizer('''def print_prime(n):
   """
   Print all primes between 1 and n
   """''', return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_length=200)
text = tokenizer.batch_decode(outputs)[0]
print(text)

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


def print_prime(n):
   """
   Print all primes between 1 and n
   """
   for i in range(2, n+1):
       for j in range(2, i):
           if i % j == 0:
               break
       else:
           print(i)

print_prime(20)
```

## Exercises

1. Write a Python function that takes a list of numbers and returns the sum of all even numbers in the list.

```python
def sum_even(numbers):
    """
    Returns the sum of all even numbers in the list
    """
    return sum(filter(lambda x: x % 2 == 0, numbers))

print(sum_even([1, 2, 3, 4, 5, 6])) # Output: 12
```

2. Write a Python function that takes


In [ ]:
!lm_eval --model hf \
    --model_args pretrained=microsoft/phi-2 \
    --tasks gsm8k \
    --device cuda:0 \
    --batch_size 16

2024-01-26:23:48:06,395 INFO     [utils.py:148] Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2024-01-26:23:48:06,395 INFO     [utils.py:160] NumExpr defaulting to 8 threads.
2024-01-26:23:48:06,654 INFO     [config.py:58] PyTorch version 2.1.0+cu121 available.
2024-01-26:23:48:06,655 INFO     [config.py:95] TensorFlow version 2.15.0 available.
2024-01-26:23:48:06,656 INFO     [config.py:108] JAX version 0.4.23 available.
2024-01-26 23:48:07.142969: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-26 23:48:07.143030: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-26 23:48:07.144746: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to

In [ ]:
!lm_eval --model hf \
    --model_args pretrained=microsoft/phi-1_5 \
    --tasks gsm8k \
    --device cuda:0 \
    --batch_size 16

2024-01-27:00:10:14,629 INFO     [utils.py:148] Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2024-01-27:00:10:14,629 INFO     [utils.py:160] NumExpr defaulting to 8 threads.
2024-01-27:00:10:14,892 INFO     [config.py:58] PyTorch version 2.1.0+cu121 available.
2024-01-27:00:10:14,892 INFO     [config.py:95] TensorFlow version 2.15.0 available.
2024-01-27:00:10:14,893 INFO     [config.py:108] JAX version 0.4.23 available.
2024-01-27 00:10:15.383830: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-27 00:10:15.383920: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-27 00:10:15.385733: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to